<a href="https://colab.research.google.com/github/JunHyeong-data/ML-DL-Study/blob/main/Basic-Deep-Learning/22_%EB%94%A5%EB%9F%AC%EB%8B%9D%2C_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5%2C_fine_tuning%2C_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning (트랜스퍼 러닝) 정리

## 1. 트랜스퍼 러닝이란?

**트랜스퍼 러닝(Transfer Learning)** 이란  
이미 **미리 학습(pretrained)** 되어 있는 뉴럴 네트워크를 가져와  
내가 원하는 데이터로 **재학습(fine-tuning)** 시키는 방법이다.

실무 및 프로덕션 환경에서는 대부분의 경우  
모델을 **처음부터 학습시키지 않고** 트랜스퍼 러닝을 사용한다.

---

## 2. 왜 트랜스퍼 러닝이 필요한가?

### 2.1 데이터 문제

- 대규모 모델을 처음부터 학습하려면 **막대한 데이터**가 필요
- 예:
  - ImageNet-1K
  - 약 **130만 장**의 이미지
- 대부분의 개인이나 기업은 이 정도 데이터를 확보하기 어렵다.

### 2.2 연산 비용 문제

- 대형 CNN 모델 학습은 **시간과 비용이 매우 큼**
- 예:
  - ResNet-152
  - 과거 세대 GPU 기준 약 **3주** 이상 학습 필요

➡️ **현실적으로 대부분은 처음부터 학습이 불가능**

---

## 3. 트랜스퍼 러닝이 잘 동작하는 이유

CNN은 학습 과정에서 다음과 같은 구조의 특징을 학습한다.

- **Low-level feature**
  - 엣지(edge)
  - 코너(corner)
  - 텍스처(texture)
- **High-level feature**
  - 물체의 형태
  - 의미 있는 패턴

ImageNet으로 학습된 모델은  
이미 **일반적인 이미지 특징을 매우 잘 학습**하고 있다.

➡️ 만약 우리가 풀고자 하는 문제가  
ImageNet과 **도메인이 비슷하다면**,  
이 특징을 그대로 재사용할 수 있다.

---

## 4. 트랜스퍼 러닝 예시

- 포켓몬 이미지 분류
- 세포 이미지 분류 (의료 이미지)
- 강아지 / 고양이 분류

이런 문제들은:
- 처음부터 학습하기엔 데이터가 부족
- 하지만 **프리트레인 모델을 사용하면 적은 데이터로도 고성능 가능**

---

## 5. 트랜스퍼 러닝 적용 전략

데이터 양과 도메인 유사성에 따라 전략이 달라진다.

---

### 5.1 ImageNet과 매우 비슷한 경우

예: 강아지 / 고양이 분류

- 마지막 **Fully Connected Layer만 학습**
- 나머지 레이어는 **freeze (고정)**

```text
[ Backbone (Frozen) ] → [ FC Layer (Train) ]
````

---

### 5.2 도메인이 다른 경우

예:

* 의료 영상 (X-ray, MRI)
* 애니메이션 이미지 (포켓몬)

이 경우:

* ImageNet과 피처 분포가 다름
* 더 많은 레이어를 학습해야 함
* 데이터도 더 많이 필요

➡️ 이 과정을 **Fine-Tuning** 이라고 함

---

### 5.3 실무 경험 기반 가이드

* 클래스당 최소 **1,000 ~ 2,000장** 데이터 권장
* 가능한 경우:

  * **모든 레이어를 fine-tuning** 하는 것이 가장 안정적
  * 여러 트릭보다 데이터 확보가 더 중요

---

## 6. 다양한 트랜스퍼 러닝 방법들

이론적으로는 매우 많은 옵션이 존재한다.

* 하위 레이어만 freeze
* 상위 레이어만 freeze
* 레이어별 다른 learning rate
* BatchNorm 레이어만 고정
* 부분 fine-tuning

➡️ 하지만 복잡도가 높고 실무에서는 관리가 어려움

---

## 7. 실습 예제 개요

### 7.1 사용 모델

* ResNet-18
* ImageNet-1K로 pretrained 된 모델

---

### 7.2 데이터셋

* CIFAR-10
* 원본 이미지 크기: 32 × 32
* ResNet 입력 요구 크기: 224 × 224

➡️ `Resize(224, 224)` 적용

---

### 7.3 데이터 전처리

* Resize
* ToTensor
* Normalize

```text
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]
```

> 이 값은 ImageNet pretrained 모델에서 사용된 값
> 데이터가 충분히 많다면 꼭 동일할 필요는 없음

---

## 8. 모델 수정

### 8.1 마지막 레이어 변경

ImageNet:

* 출력 클래스 수: 1000

CIFAR-10:

* 출력 클래스 수: 10

```python
model.fc = nn.Linear(512, 10)
```

---

### 8.2 파라미터 고정 여부

* 전체 freeze 가능
* 일부 freeze 가능
* 전체 fine-tuning 가능

이번 예제에서는:

* **모든 레이어 fine-tuning**

---

## 9. 학습 설정

* Loss Function: CrossEntropyLoss
* Optimizer: Adam
* 학습 / 검증 루프는 기존과 동일

---

## 10. 결과

* 기존 CNN:

  * Validation Accuracy ≈ 70%
* Transfer Learning 적용:

  * Validation Accuracy ≈ **95%**
* 학습 시간:

  * 약 **10 ~ 30분**

➡️ **적은 비용으로 매우 큰 성능 향상**

---

## 11. 정리

* 컴퓨터 비전에서 **Transfer Learning은 표준**
* 대부분의 CNN 모델은:

  * Pretrained Backbone + Task-specific Head 구조
* 실무에서는:

  * 모델 구조보다 **데이터 확보가 가장 중요**

---

## 12. 다음 챕터

* Object Detection
* Segmentation

CNN 기초 챕터 종료 🎉
```


In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

if torch.backends.mps.is_available():
    my_device = torch.device('mps')
elif torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')

print(my_device)

cuda


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

# Load CIFAR10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406] , std=[0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)

100%|██████████| 170M/170M [00:13<00:00, 13.1MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [4]:
from torchvision import models

weights = models.ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights)
model.fc = nn.Linear(512, 10)  # CIFAR10 has 10 classes
# print(net)

# Freeze all layers
# for param in model.parameters():
#     param.requires_grad = False

# # Unfreeze the last fully connected layer (fc)
# for param in model.fc.parameters():
#     param.requires_grad = True

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 234MB/s]


In [ ]:
import time

model.to(my_device)
num_epochs = 100
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer
    model.train()
    for batch_idx, (data, label) in enumerate(trainloader):
        data, label = data.to(my_device), label.to(my_device)
        scores = model(data)
        loss = criterion(scores, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(my_device), label.to(my_device)
            scores = model(data)
            loss = criterion(scores, label)
            val_loss += loss.item() * data.size(0)

            predicted = scores.argmax(dim=1)
            correct += predicted.eq(label).sum().item()

    val_loss /= len(testloader.dataset)
    val_accuracy = 100. * correct / len(testloader.dataset)
    end_time = time.time()  # End the timer
    elapsed_time = end_time - start_time  # Compute the elapsed time

    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%, Time: {elapsed_time:.2f}s")

Epoch [1/100], Training Loss: 0.2684, Validation Loss: 0.3739, Validation Accuracy: 88.84%, Time: 177.94s
Epoch [2/100], Training Loss: 0.1478, Validation Loss: 0.2575, Validation Accuracy: 91.90%, Time: 177.21s
Epoch [3/100], Training Loss: 0.1255, Validation Loss: 0.2167, Validation Accuracy: 93.00%, Time: 176.90s
Epoch [4/100], Training Loss: 0.1559, Validation Loss: 0.1995, Validation Accuracy: 93.29%, Time: 177.67s
Epoch [5/100], Training Loss: 0.0861, Validation Loss: 0.1898, Validation Accuracy: 93.66%, Time: 177.18s


ImageNet으로 pretrained 된 ResNet은
“224 × 224 이미지 + ImageNet 분포”를 가정하고 학습되었기 때문에
CIFAR-10을 쓸 때도 Resize((224,224))로
그 가정을 맞춰주는 것이다.

입력 이미지
 → Conv + BN + ReLU
 → Residual Block × 여러 개
 → Global Average Pooling
 → Fully Connected Layer (fc)
 → 출력
